## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-03-06-33 +0000,nypost,Knife-wielding robbery suspect on e-bike hit a...,https://nypost.com/2025/07/12/us-news/knife-wi...
1,2025-07-13-02-44-25 +0000,nypost,Border czar Tom Homan delivers fiery response ...,https://nypost.com/2025/07/12/us-news/border-c...
2,2025-07-13-02-24-58 +0000,nypost,"Twisted Nevada mom, 28, accused of tossing new...",https://nypost.com/2025/07/12/us-news/twisted-...
3,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
4,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
103,trump,12
9,nyc,4
6,killed,4
233,assassination,3
234,attempt,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...,34
29,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,32
32,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...,31
35,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...,29
8,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,34,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...
4,24,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
12,22,2025-07-12-22-31-52 +0000,nypost,Socialist NYC mayoral candidate Zohran Mamdani...,https://nypost.com/2025/07/12/us-news/mamdanis...
29,20,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
1,17,2025-07-13-02-44-25 +0000,nypost,Border czar Tom Homan delivers fiery response ...,https://nypost.com/2025/07/12/us-news/border-c...
8,17,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...
35,17,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
2,16,2025-07-13-02-24-58 +0000,nypost,"Twisted Nevada mom, 28, accused of tossing new...",https://nypost.com/2025/07/12/us-news/twisted-...
0,15,2025-07-13-03-06-33 +0000,nypost,Knife-wielding robbery suspect on e-bike hit a...,https://nypost.com/2025/07/12/us-news/knife-wi...
23,15,2025-07-12-20-22-44 +0000,nypost,"Gun-toting duo pistol-whip, rip $75K in jewelr...",https://nypost.com/2025/07/12/us-news/masked-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
